__Random Forest__

__Tiance Tan__


GOAL: Apply Random Forest to predict target variable(activity) on a 5 node m4.2xlarge cluster.

In [1]:
from pyspark.sql import Window
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml import Pipeline
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark import SparkContext
ss = SparkSession.builder.getOrCreate()


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1615517943854_0014,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f 
{
"conf":{
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
        "spark.executor.heartbeatInterval":"10800s",
        "spark.network.timeout":"24h",
        "spark.driver.memory": "20G",
        "spark.executor.memory": "20G",
        "spark.executor.cores":"8",
        "livy.server.session.timeout":"24h",
        "spark.dynamicAllocation.enabled":"false",
        "spark.ext.h2o.fail.on.unsupported.spark.param":"false",    
        "spark.app.name":"msds694"
      }
}



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1615517943854_0013,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1615517943854_0013,pyspark,idle,Link,Link,✔


In [2]:
raw = ss.read.csv("s3://msds694-group6/casas-dataset/*", header=True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def indexStringColumns(df, cols):
    
    newdf = df
    for c in cols:
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

data = raw.replace(["r2.Eat_Breakfast"], ["Eat_Breakfast"], ["activity"])\
          .replace(["r2.Personal_Hygiene"], ["Personal_Hygiene"], ["activity"])

datanumeric = indexStringColumns(data, ['activity'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_numeric = datanumeric.select(*(col(c).cast("float").alias(c) for c in raw.columns))
df_numeric.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- lastSensorEventHours: float (nullable = true)
 |-- lastSensorEventSeconds: float (nullable = true)
 |-- lastSensorDayOfWeek: float (nullable = true)
 |-- windowDuration: float (nullable = true)
 |-- timeSinceLastSensorEvent: float (nullable = true)
 |-- prevDominantSensor1: float (nullable = true)
 |-- prevDominantSensor2: float (nullable = true)
 |-- lastSensorID: float (nullable = true)
 |-- lastSensorLocation: float (nullable = true)
 |-- lastMotionLocation: float (nullable = true)
 |-- complexity: float (nullable = true)
 |-- activityChange: float (nullable = true)
 |-- areaTransitions: float (nullable = true)
 |-- numDistinctSensors: float (nullable = true)
 |-- sensorCount-Bathroom: float (nullable = true)
 |-- sensorCount-Bedroom: float (nullable = true)
 |-- sensorCount-Chair: float (nullable = true)
 |-- sensorCount-DiningRoom: float (nullable = true)
 |-- sensorCount-Hall: float (nullable = true)
 |-- sensorCount-Ignore: float (nullable = true)
 |-- sensorCount-Kitc

In [5]:
from pyspark.ml.feature import VectorAssembler

input_cols=df_numeric.columns[:-1]

va = VectorAssembler(outputCol="features", inputCols=input_cols)

df_vec = va.transform(df_numeric).select("features", "activity").withColumnRenamed("activity", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_vec.orderBy('features').show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
|(36,[0,1,2,3,4,5,...|  5.0|
+--------------------+-----+
only showing top 20 rows

In [7]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)

scalerModel = scaler.fit(df_vec)

scaledData = scalerModel.transform(df_vec).select('scaledFeatures', 'label')\
                        .withColumnRenamed("scaledFeatures", "features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.ml.feature import PCA as PCAml
from pyspark.ml.linalg import Vectors  

pca = PCAml(k=10, inputCol="features", outputCol="pca")
model = pca.fit(scaledData)
transformed = model.transform(scaledData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
transformed = transformed.select('pca', 'label').withColumnRenamed("pca", "features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
(data1, data2) = transformed.randomSplit([0.2, 0.8])
(trainingData, testData) = data1.randomSplit([0.8, 0.2])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
rf = RandomForestClassifier(maxDepth=10)

pipeline = Pipeline(stages=[rf])
evaluator = BinaryClassificationEvaluator()
paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [10,100]).build()
cv = CrossValidator(estimator=pipeline,
                    evaluator=evaluator,
                    numFolds=5,
                    estimatorParamMaps=paramGrid)

cvmodel = cv.fit(trainingData)
print(cvmodel.bestModel.stages[-1]._java_obj.parent().getNumTrees())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10

In [12]:
rfpredict = cvmodel.bestModel.transform(testData)

metric_name = "f1"
metrics = MulticlassClassificationEvaluator()\
                .setLabelCol("label")\
                .setPredictionCol("prediction")
metrics.setMetricName(metric_name)
print(np.round(metrics.evaluate(rfpredict), 4))
print()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4226

In [13]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…